## Open The Lightfield spectrometer software.

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import clr
import sys
import os
from System.IO import *
from System import String
from System.Threading import AutoResetEvent
from System.Collections.Generic import List

# Add needed dll references
sys.path.append(os.environ['LIGHTFIELD_ROOT'])
sys.path.append(os.environ['LIGHTFIELD_ROOT']+"\\AddInViews")
clr.AddReference('PrincetonInstruments.LightFieldViewV5')
clr.AddReference('PrincetonInstruments.LightField.AutomationV5')
clr.AddReference('PrincetonInstruments.LightFieldAddInSupportServices')

# PI imports
from PrincetonInstruments.LightField.Automation import Automation
from PrincetonInstruments.LightField.AddIns import CameraSettings
from PrincetonInstruments.LightField.AddIns import ExperimentSettings
from PrincetonInstruments.LightField.AddIns import DeviceType
from PrincetonInstruments.LightField.AddIns import SpectrometerSettings

        
def save_file(filename):    
    # Set the base file name
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationBaseFileName,
        Path.GetFileName(filename))
    
    # Option to Increment, set to false will not increment
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachIncrement,
        True)

    # Option to add date
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachDate,
        True)

    # Option to add time
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachTime,
        True)

def device_found():
    # Find connected device
    for device in experiment.ExperimentDevices:
        if (device.Type == DeviceType.Camera):
            return True
     
    # If connected device is not a camera inform the user
    print("Camera not found. Please add a camera and try again.")
    return False  



# Create the LightField Application (true for visible)
# The 2nd parameter forces LF to load with no experiment 
auto = Automation(True, List[String]())
application= auto.LightFieldApplication
experiment = auto.LightFieldApplication.Experiment
file_manager=application.FileManager

## Take Background pictures at different wavelength numbers: 1500 and 2700. They are named ase background1D.csv and background2D.csv

In [ ]:

def set_value(setting, value):    
# Check for existence before setting
# gain, adc rate, or adc quality
    if experiment.Exists(setting):
        experiment.SetValue(setting, value)

def experiment_completed(sender, event_args):    
    print("Experiment Completed")    
    # Sets the state of the event to signaled,
    # allowing one or more waiting threads to proceed.
    acquireCompleted.Set()

import time

# Check for device and inform user if one is needed
if (device_found()==True): 
    experiment.ExperimentCompleted += experiment_completed 
    # Check this location for saved spe after running
    #print("Please Enter the Exposure Time:\n")
    #x=int(input())
    set_value(CameraSettings.ShutterTimingExposureTime,3000)
    #print("Please Enter the Number of Frames")
    #y=int(input())
    experiment.SetValue(ExperimentSettings.AcquisitionFramesToStore,5)
    for k in range(1,3):
        if k==1:
            experiment.SetValue(SpectrometerSettings.GratingCenterWavelength,578.1351880026082)
        elif k==2:
            experiment.SetValue(SpectrometerSettings.GratingCenterWavelength,621.2340604418703)
        _file_name = "case"

        # Pass location of saved file
        save_file(_file_name)

        # Acquire image
        experiment.Acquire()
        time.sleep(35)
        directory="C:\\Users\\labuser\\Desktop\\data\\Raman\\Vivek\\2019-10-08"
        if( os.path.exists(directory)):        
                print("\nFound the .spe file...")        

                # Returns all .spe files
                files = glob.glob(directory +'/*.spe')

                # Returns recently acquired .spe file
                last_image_acquired = max(files, key=os.path.getctime)

                try:
                    # Open file
                    file_name = file_manager.OpenFile(
                        last_image_acquired, FileAccess.Read)

                    # Access image
                    file=file_name
                    imageData = file.GetFrame(0,0)
                    #here is a problem 11-18-2019
                    n=5
                    intensity_frame=np.zeros((n,1340))
                    # Get image data
                    buffer = imageData.GetData()
                    #buffer=imageData.GetDataBuffer()
                    # Print first 10 pixel intensities
                    for i in range(0,n):
                        imageData=file.GetFrame(0,i)
                        buffer=imageData.GetData()
                        for pixel in range(0,1340):
                            intensity_frame[i][pixel]=buffer[pixel]

                    file_name.Dispose()


                except IOError:
                    print ("Error: can not find file or read data")

        else:
            print(".spe file not found...")

        print(String.Format("{0} {1}",
                            "Image saved to",
                            experiment.GetValue(
                                ExperimentSettings.
                                FileNameGenerationDirectory)))  


        wl= experiment.SystemColumnCalibration
        wavelength=np.zeros((1,1340))
        for i in range(1340):wavelength[0,i]=wl[i]
        #print(intensity_frame)
        intensity=np.zeros((1,1340))
        for i in range(1340):
            x=0
            for j in range(n):
                x=x+intensity_frame[j][i]
            x=x/n
            intensity[0,i]=x
            check_intensity=x

        w=[]
        inten=[]

        for x in range(1340):
            wavelength[0,x]=1e7*(1/532 - 1/wavelength[0,x])
            w.append(wavelength[0,x])
            inten.append(intensity[0,x])
        import csv

        m="background"+str(k)+"D.csv"
        with open(m, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["W", "I"])
            writer.writerows(zip(w,inten))

